<span style="font-size:24pt; color:blue; font-family: 'Times New Roman'">P-box Notebook 
    <span style="color:green;"> for creating AR6 style P-box data </span>
</span>
<br>
<span style="font-size:12pt; color:black; font-family:Georgia, serif;font-style:italic">by Praveen Kumar and Robert Kopp.</span>
<br>
<br>
<span style="font-family: 'Times New Roman'; font-size:16pt; color:black"> 
    This is a notebook that can be used to create and vizualize P-box data from raw <a href="https://doi.org/10.5194/egusphere-2023-14" style="color:blue; text-decoration:underline;">FACTS 1.0</a>
    output. Workflows and associated *.nc files are created as part of this notebook. 
<br>
<br>
<span style="font-family:'Courier New', monospace; font-size:12pt; color:black"> <b>Background::</b> <br>
    In order to represent deep uncertainty, Probability boxes (p-boxes) can be constructed. These represent an envelope encompassing alternative probability distributions for describing the same variables. Within this notebook, we construct four p-boxes: <strong>1e, 1f, 2e,</strong> and <strong>2f</strong>.
    <br><br>
    <strong>P-box 1e</strong> and <strong>2e</strong> use ice-sheet projections from an emulator that accurately captures the output of both the GlacierMIP glacier model intercomparison and the ISMIP6 ice-sheet model intercomparison.
    <br><br>
    <strong>P-box 1e</strong> and <strong>1f</strong> both use multi-model intercomparison exercises 
    (<a href="https://www.nature.com/articles/s41586-021-03302-y" style="color:blue; text-decoration:underline;">ISMIP6</a> and <a href="https://doi.org/10.5194/esd-11-35-2020" style="color:blue; text-decoration:underline;">LARMIP2</a>). 
    These p-boxes represent a medium-to-high level of agreement among the participating models. The IPCC AR6 thus assessed medium confidence in their projections of included processes.
</span>
<br><br><br>
    
    
<span style="font-size:14pt; color:blue"> ==> Code structure (... working on optimizing and removing redundancies to make user friendly) </span>

<ul style="font-size:12pt; color:black"> 
Make sure to set the <b>expFolder</b> to point to your facts exp output folder. This notebook creates the following folders within the current directory. 
<li></li>
<li> 1_workflow  </li>
<li> 3_pbox </li>
<li> 4_confidence_level_files </li>
</ul>


In [1]:
import shutil
import os
PD=os.getcwd()
import fun_pbox_Gen_v1 as fn
#
ssps=["ssp119","ssp126","ssp245","ssp370","ssp585"]
#
workflow = ["wf_1e","wf_1f","wf_2e","wf_2f","wf_3e", "wf_3f", "wf_4"]
#
expFolder="/projects/kopp/facts-experiments/221217"
#
region = 'global'

<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate:: <br>
    <ol style="font-size: 12pt; color: green; font-family: 'Times New Roman'">
    <li> Workflow folders</li>
    <li> Quantile files</li>
</ol>  
</span>


In [2]:
def copy_and_convert(source_file, dest_folder, region):
    #shutil.copy2(source_file, dest_folder)
    out_file_path = os.path.join(dest_folder, os.path.basename(source_file).split(".nc")[0] + "_quantiles.nc")
    fn.Samples_to_Quantiles(source_file, out_file_path)

path = os.path.join(os.getcwd(), "1_workflow")
shutil.rmtree(path, ignore_errors=True)
os.makedirs(path)

skip = {
    'wf3e': ['ssp119', 'ssp370'],
    'wf3f': ['ssp119', 'ssp370'],
    'wf4':  ['ssp119', 'ssp245', 'ssp370']
}

for workflow, file_patterns in fn.WF_file_patterns.items():
    for ssp in ssps:
        if ssp in skip.get(workflow, []):
            continue
        
        ssp_path = os.path.join(path, workflow, ssp)
        os.makedirs(ssp_path, exist_ok=True)
        
        # Copy Component files & convert to Quantiles.
        for file in file_patterns:
            component_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{file}_{region}sl.nc'
            copy_and_convert(component_file_path, ssp_path, region)
        
        # Copy Common files & convert to Quantiles.
        for common_file_item in fn.common_files:
            common_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{common_file_item}_{region}sl.nc'
            copy_and_convert(common_file_path, ssp_path, region)
        
        # Copy Total files & convert to Quantiles.
        total_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.total.workflow.{workflow}.{region}.nc'
        copy_and_convert(total_file_path, ssp_path, region)
        
        # Copy VLM file & convert to Quantiles.  
        if region == 'local':
            vlm_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{fn.local_files}.{region}sl.nc'
            copy_and_convert(vlm_file_path, ssp_path, region)


<span style="font-size: 32pt; color: green; font-family: 'Times New Roman'"> 
<br>
    check:: a sample data set
</span>

In [3]:
import xarray as xr
dum_data_path='./1_workflow/wf1e/ssp119/coupling.ssp119.emuAIS.emulandice.AIS_globalsl_quantiles.nc'
dum_data_nc=xr.open_dataset(dum_data_path)
dum_data_nc

<xarray.Dataset>
Dimensions:           (quantiles: 107, years: 9, locations: 1)
Coordinates:
  * years             (years) int32 2020 2030 2040 2050 2060 2070 2080 2090 2100
  * locations         (locations) int64 -1
  * quantiles         (quantiles) float64 0.0 0.001 0.005 ... 0.995 0.999 1.0
Data variables:
    sea_level_change  (quantiles, years, locations) float32 ...
    lat               (locations) float32 ...
    lon               (locations) float32 ...
Attributes:
    description:  Global SLR contribution from Antarctica using the emulandic...
    history:      Created Sat Dec 17 11:01:00 2022
    source:       FACTS: coupling.ssp119.emuAIS.emulandice.AIS. 
    baseyear:     2005
    scenario:     ssp119

<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate::  <span style="color: blue;"> P-box 
</span></span>

In [ ]:
base_path = os.getcwd()
path = os.path.join(base_path, "3_pbox")
shutil.rmtree(path, ignore_errors=True)
os.makedirs(path)
#
#
def handle_pbox(ssp_path, pbox, ssp, workflow_suffix):
    # AIS Pbox component
    fname1 = fn.find_filename_with_pattern(ssp_path, f"{workflow_suffix}.AIS")
    fname2 = fn.find_filename_with_pattern(ssp_path, "larmip.AIS")
    infiles = [os.path.join(ssp_path, fname) for fname in [fname1, fname2]]
    outfile = os.path.join(ssp_path, f"icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc")
    fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
    
    # total
    source_dir = os.path.join(base_path, f'1_workflow/wf{workflow_suffix}/{ssp}')
    fn.copy_filename_with_pattern(source_dir, ssp_path, "*total.workflow*")
    
    fname1 = fn.find_filename_with_pattern(ssp_path, f"total.workflow.wf{workflow_suffix[3]}")
    fname2 = fn.find_filename_with_pattern(ssp_path, f"total.workflow.wf2{workflow_suffix[3]}")
    infiles = [os.path.join(ssp_path, fname) for fname in [fname1, fname2]]
    outfile = os.path.join(ssp_path, 'total-workflow.nc')
    fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
    
    fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
    fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

for pbox in fn.PB_file_patterns:
    pbox_path = os.path.join(path, pbox)
    os.makedirs(pbox_path, exist_ok=True)
    
    for ssp in ssps:
        if pbox in ['pb_2e', 'pb_2f'] and ssp in ['ssp119','ssp245','ssp370']:
            continue
        
        ssp_path = os.path.join(pbox_path, ssp)
        os.makedirs(ssp_path, exist_ok=True)
        
        if pbox in ['pb_1e', 'pb_1f']:
            # Copy all WF files.
            source_dir = os.path.join(base_path, f'1_workflow/wf{pbox.split("_")[1]}/{ssp}')
            fn.copy_all_files_from(source_dir, ssp_path) 
            
            # We need larmipAIS (common to pb_1e & pb_1f)
            source_dir = os.path.join(base_path, f'1_workflow/wf2e/{ssp}')
            fn.copy_filename_with_pattern(source_dir, ssp_path, "*larmip.AIS*")
            
            if pbox == 'pb_1e':
                handle_pbox(ssp_path, pbox, ssp, "emulandice")
                
            if pbox == 'pb_1f':
                handle_pbox(ssp_path, pbox, ssp, "ipccar5.icesheets")


<br>
<span style="font-size: 32pt; color: green; font-family: 'Times New Roman'"> 
    Shrink code below to functions
</span>

In [4]:
path = os.path.join(os.getcwd(), "3_pbox")
shutil.rmtree(path, ignore_errors=True)
os.makedirs(path)
#
for pbox in fn.PB_file_patterns:
    os.makedirs(os.path.join(path, pbox), exist_ok=True)
    for ssp in ssps:
        # ..............................................................................
        if pbox in ['pb_2e', 'pb_2f'] and ssp in['ssp119','ssp245','ssp370']: continue
        # ..............................................................................
        ssp_path = os.path.join(path, pbox,ssp)
        os.makedirs(ssp_path, exist_ok=True)
        #
        # 
        if pbox in ['pb_1e','pb_1f']:
            # Copy all WF files.
            source_dir=f'1_workflow/wf{pbox.split("_")[1]}/{ssp}'
            source_dir_path=os.path.join(os.getcwd(),source_dir)
            fn.copy_all_files_from(source_dir_path,ssp_path) 
            #
            # We need larmipAIS (common to pb_1e & pb_1f)
            source_dir=f'1_workflow/wf2e/{ssp}'
            source_dir_path=os.path.join(os.getcwd(),source_dir)
            fn.copy_filename_with_pattern(source_dir_path,ssp_path,"*larmip.AIS*")
            #
            #
            if pbox in ['pb_1e']:
                #
                # AIS Pbox component
                fname1=fn.find_filename_with_pattern(ssp_path,"emulandice.AIS")
                fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                #
                outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                # total
                source_dir=f'1_workflow/wf2e/{ssp}'
                source_dir_path=os.path.join(os.getcwd(),source_dir)    
                fn.copy_filename_with_pattern(source_dir_path,ssp_path,"*total.workflow*")
                #
                fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1e");               
                fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2e")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      
                outfile=f'{ssp_path}/total-workflow.nc'
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")
            #
            #
            if pbox in ['pb_1f']:
                #
                # AIS Pbox component.
                fname1=fn.find_filename_with_pattern(ssp_path,"ipccar5.icesheets_AIS")
                fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                #
                outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                # total
                source_dir=f'1_workflow/wf2f/{ssp}'
                source_dir_path=os.path.join(os.getcwd(),source_dir) 
                fn.copy_filename_with_pattern(source_dir_path,ssp_path,"*total.workflow*")
                #
                fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1f");               
                fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2f")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      
                outfile=f'{ssp_path}/total-workflow.nc'
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

In [ ]:

        if indv_Pbox1 in ['pb_2e']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            # emulandice.AIS
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*emulandice.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.glaciers*");
            
            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # Glaciers Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.glaciers")
            fname2=fn.fileNAME0(ssp_path,"ipccar5.glaciers")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"glaciers-pb"+indv_Pbox1.split('_')[1]+"-glaciers-"+ssp1+"_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  
            
            # total
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2e");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1e"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3e"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*glaciers*", "glaciers-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

        if indv_Pbox1 in ['pb_2f']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.icesheets_AIS*");
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");


            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"GrIS1f.FittedISMIP.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"ipccar5.icesheets_AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  

            # total
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2f");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1f"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3f"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

In [ ]:
# Pbox = ["pb_1e","pb_1f","pb_2e","pb_2f"]
# #
# # Create a folder
# Pbox_path = fn.FOcr8("3_pbox")
# #
# for indv_Pbox0,indv_Pbox1 in enumerate(Pbox):
#     pb_path=fn.FOcr8_1(Pbox_path, indv_Pbox1) #create wf folder
#     for ssp0,ssp1 in enumerate(ssp):
#         # ..........................................................................................
#         if indv_Pbox1 in ['pb_2e', 'pb_2f'] and ssp1 in['ssp119','ssp245','ssp370']: continue
#         # ..........................................................................................
#         ssp_path=fn.FOcr8_1(pb_path, ssp1) #create ssp folders within the pb
#         #
        if indv_Pbox1 in ['pb_1e','pb_1f']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            # larmipAIS (common to pb_1e & pb_1f)
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_2e/"+ssp1; fn.cp(srcDIR,ssp_path,"*larmip.AIS*")
            #
            if indv_Pbox1 in ['pb_1e']:
                # AIS Pbox component
                fname1=fn.fileNAME0(ssp_path,"emulandice.AIS")
                fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                # total
                srcDIR=PD+"/2_workflow_quantiles/"+"wf_2e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
                fname1=fn.fileNAME0(ssp_path,"total.workflow.wf1e");               fname2=fn.fileNAME0(ssp_path,"total.workflow.wf2e")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      outfile=ssp_path+"/total-workflow.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

                #
            if indv_Pbox1 in ['pb_1f']:
                # AIS Pbox component.
                fname1=fn.fileNAME0(ssp_path,"ipccar5.icesheets_AIS")
                fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                # total
                srcDIR=PD+"/2_workflow_quantiles/"+"wf_2f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
                fname1=fn.fileNAME0(ssp_path,"total.workflow.wf1f");               fname2=fn.fileNAME0(ssp_path,"total.workflow.wf2f")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      outfile=ssp_path+"/total-workflow.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")


        
        if indv_Pbox1 in ['pb_2e']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            # emulandice.AIS
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*emulandice.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.glaciers*");
            
            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # Glaciers Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.glaciers")
            fname2=fn.fileNAME0(ssp_path,"ipccar5.glaciers")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"glaciers-pb"+indv_Pbox1.split('_')[1]+"-glaciers-"+ssp1+"_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  
            
            # total
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2e");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1e"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3e"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*glaciers*", "glaciers-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

        if indv_Pbox1 in ['pb_2f']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.icesheets_AIS*");
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");


            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"GrIS1f.FittedISMIP.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"ipccar5.icesheets_AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  

            # total
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2f");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1f"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3f"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

---
## <font color=red> Generate:: 006_CI-level_OP_files  </font>
---

In [ ]:
pboxdir=PD+'/3_pbox'
outdir = fn.FOcr8("4_confidence_level_files")
fn.GenerateConfidenceFiles(pboxdir, outdir)